In [39]:
from skimage import io
from skimage.transform import resize
import numpy as np
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
import math

def imgToBinaryMatrix(image):
    image = resize(image, (20, 20), preserve_range=True).astype('uint8')

    # specify a threshold 0-255
    threshold = 160

    # make all pixels < threshold black
    binarized = 1.0 * (image > threshold)
    
    #convert 3d array to 2d array
    return binarized[:,:,0]


In [40]:
def convertMatrixTo3x3(matrix):
    xs = matrix.shape[0]//3  # division lines for the picture
    ys = matrix.shape[1]//3

    # now slice up the image (in a shape that works well with subplots)
    newMatrix = [[matrix[0:xs, 0:ys], matrix[0:xs, ys:ys*2], matrix[0:xs, ys*2:ys*3]], 
    [matrix[xs:xs*2, 0:ys], matrix[xs:xs*2, ys:ys*2], matrix[xs:xs*2, ys*2:ys*3]], 
    [matrix[xs*2:xs*3, 0:ys], matrix[xs*2:xs*3, ys:ys*2], matrix[xs*2:xs*3, ys*2:ys*3]], 
    ]
    return newMatrix

def convertMatrixTo4x4(matrix):
    xs = matrix.shape[0]//4  # division lines for the picture
    ys = matrix.shape[1]//4

    # now slice up the image (in a shape that works well with subplots)
    newMatrix = [[matrix[0:xs, 0:ys], matrix[0:xs, ys:ys*2], matrix[0:xs, ys*2:ys*3], matrix[0:xs, ys*3:ys*4]], 
    [matrix[xs:xs*2, 0:ys], matrix[xs:xs*2, ys:ys*2], matrix[xs:xs*2, ys*2:ys*3], matrix[xs:xs, ys*3:ys*4]], 
    [matrix[xs*2:xs*3, 0:ys], matrix[xs*2:xs*3, ys:ys*2], matrix[xs*2:xs*3, ys*2:ys*3], matrix[xs*2:xs*3, ys*3:ys*4]], 
    [matrix[xs*3:xs*4, 0:ys], matrix[xs*3:xs*4, ys:ys*2], matrix[xs*3:xs*4, ys*3:ys*4], matrix[xs*3:xs*4, ys*3:ys*4]]
    ]
    return newMatrix


def convert3x3ToVector(matrix):
    vector = []
    for i in range(3):
        for j in range(3):
            vector.append(np.count_nonzero(matrix[i][j] == 0))
    return vector

def convert4x4ToVector(matrix):
    vector = []
    for i in range(4):
        for j in range(4):
            vector.append(np.count_nonzero(matrix[i][j] == 0))
    return vector

"""
fig, axs = plt.subplots(3, 3)
vector = []
for i in range(3):
    for j in range(3):
       axs[i][j].imshow(splits[i][j])
"""

def euclideanDistance(v1, v2):
    distance = 0
    for i in range(len(v1)):
        distance += math.pow((v1[i] - v2[i]),2)
    return math.sqrt(distance)
    
def getComparedVectorsExcept(number, nth):
    vectors = []
    for i in range(10):
        for j in range(1,11):
            if i == number and j == nth:
                    continue
            name = f'baseProjetOCR/{i}_{j}.png'
            matrixImg = imgToBinaryMatrix(io.imread(name))
            matrixImg = convertMatrixTo4x4(matrixImg)
            vectors.append((name,convert4x4ToVector(matrixImg)))
    return vectors

def getFileNameCorrespondingTo(img, comparedVectors):
    matrixImg = imgToBinaryMatrix(img)
    matrixImg = convertMatrixTo4x4(matrixImg)
    sourceVector = convert4x4ToVector(matrixImg)

    allDistancesZoning = [euclideanDistance(sourceVector, comparedVectors[i][1]) for i in range(len(comparedVectors))]
    minDistZoning = min(allDistancesZoning)

    index = allDistancesZoning.index(minDistZoning)
    return comparedVectors[index][0]

def getNumberOfImage(s):
    return int(s.split("/",1)[1][0])

def confusionMatrix():
    mat = []
    for i in range(10):
        vectorTmp = [0]*10
        for j in range(11):
            if j == 0:
                continue
            vectors = getComparedVectorsExcept(i, j)
            n = getNumberOfImage(getFileNameCorrespondingTo(io.imread(f'baseProjetOCR/{i}_{j}.png'), vectors))
            vectorTmp[n] += 1
        mat.append(vectorTmp)
    return mat

confMatrix = confusionMatrix()
score = sum([confMatrix[i][i] for i in range(10)])
print(score)
print(confMatrix)


[13.228756555322953, 12.68857754044952, 14.696938456699069, 13.2664991614216, 19.4164878389476, 25.98076211353316, 19.183326093250876, 24.289915602982237, 19.72308292331602, 31.192947920964443, 34.45286635390443, 23.130067012440755, 26.94438717061496, 32.60368077380221, 33.391615714128, 28.42534080710379, 29.949958263743873, 33.46640106136302, 36.05551275463989, 28.74021572639983, 24.71841418861655, 28.106938645110393, 27.51363298439521, 25.11971337416094, 22.561028345356956, 27.568097504180443, 30.215889859476256, 25.768197453450252, 25.099800796022265, 29.410882339705484, 31.38470965295043, 31.52776554086889, 30.773365106858236, 32.14031735997639, 28.653097563788805, 23.15167380558045, 24.576411454889016, 27.110883423451916, 27.147743920996454, 34.438350715445125, 33.36165463522455, 33.075670817082454, 25.787593916455254, 35.185224171518364, 29.832867780352597, 32.78719262151, 40.50925820105819, 31.63858403911275, 31.176914536239792, 23.83275057562597, 23.979157616563597, 24.04163056